In [ ]:
from __future__ import print_function

# Traitement de données classique

Nous allons voir dans cet exemple comment utiliser la bibliothèque `numpy` pour récupérer des valeurs dans un fichier csv et commencer à les traiter.
Nous allons utiliser le module `csv` qui permet de lire un fichier csv et d'en extraire les valeurs lignes par lignes.

Nous allons travailler sur le fichier de données d'entraînement du Titanic. Le but est de prédire les chance de survie à bord du bateau. Il faut récupérer le fichier `train.csv` (voir le premier cours ou téléchargez le depuis https://www.kaggle.com/c/titanic-gettingStarted/data ) et le sauvegarder dans le répertoire dans lequel le notebook s'éxecute. Vous pouvez utiliser la commande **`pwd`** pour connaître ce répertoire. Sinon, vous pouvez déplacer le répertoire courant pour rejoindre l'endroit où vous avez sauvegardé votre fichier avec la commande **`cd`**.

In [ ]:
import csv
import numpy as np

fichier_csv = csv.reader(open('train.csv', 'r'))
entetes = fichier_csv.next()  # on récupère la première ligne qui contient les entetes
donnees = list()              # on crée la liste qui va servir à récupérer les données

for ligne in fichier_csv:     # pour chaque ligne lue dans le fichier csv
    donnees.append(ligne)     # on ajoute les valeurs lues dans le tableau donness
donnees = np.array(donnees)   # le tableau donnees est transformé en numpy array

Regardons comment sont stockées les données en mémoire:

In [ ]:
print (donnees)

Regardons maintenant la colonne de l'âge, n'affichons que les 15 premières valeurs:

In [ ]:
print (donnees[0:15, 5])

On peut donc remarquer que les âges sont stockés comme des chaîne de caractères. Transformons les en réels :

In [ ]:
age = donnees[0:15, 5].astype(np.float)

Numpy ne sait pas convertir la chaîne de caractère vide `''` (en 6e position dans notre liste) en réels. Pour traiter ces données, il faudrait écrire un petit algorithme. Nous allons voir comment on peut utiliser `pandas` pour faire ces traitements beaucoup plus facilement.

# Traiter et manipuler les données avec `pandas`

In [ ]:
import pandas as pd
import numpy as np

Pour lire le fichier csv nous allons utiliser la fonction **`read_csv`**

In [ ]:
df = pd.read_csv('train.csv')

Pour vérifier si cela a bien fonctionné, affichons les premières valeurs. On voit apparaître l'identifiant du passager, s'il a survécu, sa classe, son nom, son sexe, son âge, le nombre de frères/soeurs/époux/épouse sur le bâteau, le nombre de parents ou d'enfants, le numéro de ticket, le prix, le numéro de cabine et le port d'embarquement. 

In [ ]:
df.head(5)

Comparons le type de `donnees`, obtenu précédemment. C'est un numpy array. Le type de `df` est un objet spécifique à `pandas`.

In [ ]:
type(donnees)

In [ ]:
type(df)

Nous avions vu qu'avec `numpy`, toutes les valeurs importées étaient des chaînes de caractères. Vérifions ce qu'il en est avec `pandas`

In [ ]:
df.dtypes

On peut voir que `Pandas` a détecté automatiquement le types des données de notre fichier csv: soit des entiers, soit des réels, soit des objets (chaînes de caractères). Il y a deux commandes importantes à connaître, c'est **`df.info()`** et **`df.describe()`** 

In [ ]:
df.info()

L'âge n'est pas renseigné pour tous les passagers, seulement pour 714 passagers sur 891. Idem pour le numéro de cabine et le port d'embarquement. On peut également utiliser **`describe()`** pour calculer plusieurs indicateurs statistiques utiles.

In [ ]:
df.describe()

On peut voir que `pandas` a calculé automatiquement les indicateurs statistiques en tenant compte uniquement des données renseignées. Par exemple, il a calculé la moyenne d'âge uniquement sur les 714 valeurs connues. `pandas` a laissé de coté les valeurs non-numériques (nom, sexe, ticket, cabine, port d'embarquement).

# Pour aller un peu plus loin avec pandas

## Référencement et filtrage

Pour afficher uniquement les 10 premières valeurs de la colonne âge :

In [ ]:
df['Age'][0:10]

On peut également utiliser la syntaxe

In [ ]:
df.Age[0:10]

*A faire à la maison* afficher le les 10 premières valeurs de la colonne des `'Cabin'`.

On peut calculer des critères statistiques directement sur les colonnes

### df.Age.mean()

On peut voir que c'est la même valeur que celle affichée dans **`describe`**. Cette syntaxe permet d'utiliser facilement la valeur de la moyenne dans des calculs ou des algorithmes.

Pour filtrer les données, on va passer la liste de colonnes désirées:

In [ ]:
colonnes_interessantes = ['Sex', 'Pclass', 'Age']
df[ colonnes_interessantes ]

En analyse, on est souvent intéressé  par filtrer les données en fonction de certains critères. Par exemple, l'âge maximum est 80 ans. On peut examiner les informations relatives aux personnes âgées :

In [ ]:
df[df['Age'] > 60]

Comme on a trop d'informations, on peut les filtrer:

In [ ]:
df[df['Age'] > 60][['Pclass', 'Sex', 'Age', 'Survived']]

On peut voir que parmis les persones âges, il y a principalement des hommes. Les personnes qui   ont survécues était principalement des femmes. 

Nous allons maintenant voir comment traiter les valeurs manquantes pour l'âge. Nous allons filtrer les données pour afficher uniquement les valeurs manquantes

In [ ]:
df[df.Age.isnull()][['Sex', 'Pclass', 'Age']]

Pour combiner des filtres, on peut utiliser `'&'`. Affichons le nombre d'hommes dans chaque classe

In [ ]:
for i in range(1, 4):
    print ("Dans la classe", i, ", il y a", len( df[ (df['Sex'] == 'male') & (df['Pclass'] == i) ]), "hommes")

Visualisons maintenant l'histogramme de répartition des âges.

In [ ]:
df.Age.hist(bins=20, range=(0,80))

# Créations et modifications des colonnes

Pour pouvoir exploiter les informations sur le sexe des personnes, nous allons ajouter une nouvelle colonne, appellée *genre*, qui vaudra 1 pour les hommes et 0 pour les femmes.

In [ ]:
df['Gender'] = 4 # on ajoute une nouvelle colonne dans laquelle toutes les valeurs sont à 4
df.head()

In [ ]:
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ) # la colonne Gender prend 0 pour les femmes et 1 pour les hommes
df.head()

Pour créer et renommer de nouvelles colonnes, on peut également agréger des informations issues de différentes colonnes. Créons par exemple une colonne pour stocker les nombre de personnes de la même famille à bord du Titanic.

In [ ]:
df['FamilySize'] = df.SibSp + df.Parch
df.head()

Nous allons remplir les valeurs manquantes de l'âge avec la valeur médiane dépendant de la classe et du sexe.

In [ ]:
ages_medians = np.zeros((2, 3))
ages_medians

In [ ]:
for i in range(0,2):
    for j in range(0,3):
        ages_medians[i,j] = df[ (df['Gender'] == i) & (df['Pclass'] == j+1) ]['Age'].median()
        
ages_medians

On va créer une nouvelle colonne AgeFill qui va utiliser ces âges médians

In [ ]:
for i in range(0, 2):
    for j in range (0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1), 'AgeFill'] = ages_medians[i,j]

# pour afficher les 10 premières valeurs qui sont complétées
df [df.Age.isnull()][['Gender', 'Pclass', 'Age', 'AgeFill']].head(10)

Pour sauvegarder votre travail, vous pouvez utiliser le module `pickle` qui compresse et sauvegarde vos données :

In [ ]:
import pickle

f = open('masauvegarde.pck', 'w')
pickle.dump(df, f)
f.close()

Pour récuperer votre travail, on utilise l'opération inverse, toujours avec `pickle`

In [ ]:
with open('masauvegarde.pck', 'r') as f:
    dff = pickle.load(f)

Ce notebook est une adaptation de celui proposé sur la page kaggle : https://www.kaggle.com/c/titanic-gettingStarted/details/getting-started-with-python-ii